# Rotina para fazer suavização - IPDIO atividade avaliativa

Projeto Carina, Fernanda e Jeandria

In [3]:
#####Funções de suavização com MetPy
# Baseado no arquivo Smoothing.py de:
# https://unidata.github.io/MetPy/latest/examples/calculations/Smoothing.html
#
#####

import pygrib
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import pygrib
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy.io.shapereader as shpreader
import metpy.calc as mpcalc
from itertools import product
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

AttributeError: module 'numpy' has no attribute 'ndarray'

In [ ]:
#### Buscando dados de vorticidade e pressão do ERA5:
#Abrindo os arquivos:
file = 'dados_grib/nivelpress/vorticidade_20200701.grib'
grbs = pygrib.open(file)

In [ ]:
grbs.seek(0)

In [ ]:
#Abrindo as variáveis:
grb_vort_850_t0 = grbs.select(name='Vorticity (relative)',level=850)[18]
vort_850 = grb_vort_850_t0.values
#Para o valor de vorticidade em unidade mais amigavel, multiplicar por 10000:
vort_850a = 10000*vort_850
#Buscando a latitude e longitude do arquivo do ERA5:
for g in grbs:
    lats, lons = g.latlons()
    lats, lons = np.array(lats), np.array(lons)

In [ ]:
####Plotar com mais informações geográficas:

# Para colocar os estados no mapa:
estados_br= 'estados_Brasil/estados_2010.shp'
adm1_shapes = list(shpreader.Reader(estados_br).geometries())
# Informações sobre os limites do mapa:
extent = [-60, -30, -20, -50]

In [ ]:
# Na projeção PlateCarree:
ax = plt.axes(projection=ccrs.PlateCarree())
ax.add_geometries(adm1_shapes,ccrs.PlateCarree(), edgecolor='black', facecolor='none', alpha=0.5)
ax.add_feature(cfeature.LAND)
ax.add_feature(cfeature.LAKES)
ax.gridlines(crs=ccrs.PlateCarree(),
       draw_labels=True,linewidth=0.5,color='gray',
       alpha=0.5,linestyle='--')
ax.xformatter = LONGITUDE_FORMATTER
ax.yformatter = LATITUDE_FORMATTER
ax.set_extent(extent)

In [ ]:
# Coloca um fundo legal com relevo e oceano:
#ax.stock_img()
ax.add_feature(cfeature.BORDERS)
#ax.gridlines()
ax.coastlines(resolution='50m')

In [ ]:
# Plotar vorticidade no mapa:
vortic_850 = ax.contourf(lons,lats,vort_850a,levels=10, vmax=-1,cmap='hot')

In [ ]:
# Coloca o titulo:
plt.title('Vorticidade em 850 hPa \n em 01/07/2020, 18 horas')
# Coloca a barra lateral de cores da vorticidade:
plt.colorbar(vortic_850)
# Coloca a informacao em um arquivo:
plt.savefig('vort850_s_suav_20200701_18.png')
print('Campo de vorticidade sem suavização')
plt.show()

# Metodos de suavizacao

In [ ]:
# Para colocar os estados no mapa:
estados_br= 'estados_Brasil/estados_2010.shp'
adm1_shapes = list(shpreader.Reader(estados_br).geometries())
# Informações sobre os limites do mapa:
extent = [-60, -30, -20, -50]

In [ ]:
# Na projeção PlateCarree:
ax = plt.axes(projection=ccrs.PlateCarree())
ax.add_geometries(adm1_shapes,ccrs.PlateCarree(), edgecolor='black', facecolor='none', alpha=0.5)
ax.add_feature(cfeature.LAND)
ax.add_feature(cfeature.LAKES)
ax.gridlines(crs=ccrs.PlateCarree(),
       draw_labels=True,linewidth=0.5,color='gray',
       alpha=0.5,linestyle='--')
ax.xformatter = LONGITUDE_FORMATTER
ax.yformatter = LATITUDE_FORMATTER
ax.set_extent(extent)

In [ ]:
# Coloca um fundo legal com relevo e oceano:
#ax.stock_img()
ax.add_feature(cfeature.BORDERS)
ax.coastlines(resolution='50m')

Gaussian smoother

In [ ]:
############### Gaussian Smoother ###############
#gaussian1 = mpcalc.smooth_gaussian(vort_850a, 3)
gaus1 = ax.contourf(lons,lats,mpcalc.smooth_gaussian(vort_850a, 3),levels=10, vmax=-1,cmap='hot')
plt.title('Vorticidade em 850 hPa \n em 01/07/2020, 18 horas\n Gaussian - Low Degree')
# Coloca a barra lateral de cores da vorticidade:
plt.colorbar(gaus1)
# Coloca a informacao em um arquivo:
plt.savefig('vort850_gaus1_20200701_18.png')
print('Campo de vorticidade com suavização - Gaussian - Low Degree')
#axa[0, 0].imshow(mpcalc.smooth_gaussian(vort_850a, 3))
#axa[0, 0].set_title('Gaussian - Low Degree')
plt.show()


In [ ]:
####Plotar com mais informações geográficas:

# Para colocar os estados no mapa:
estados_br= 'estados_Brasil/estados_2010.shp'
adm1_shapes = list(shpreader.Reader(estados_br).geometries())
# Informações sobre os limites do mapa:
extent = [-60, -30, -20, -50]
# Na projeção PlateCarree:
ax = plt.axes(projection=ccrs.PlateCarree())
ax.add_geometries(adm1_shapes,ccrs.PlateCarree(), edgecolor='black', facecolor='none', alpha=0.5)
ax.add_feature(cfeature.LAND)
ax.add_feature(cfeature.LAKES)
ax.gridlines(crs=ccrs.PlateCarree(),
       draw_labels=True,linewidth=0.5,color='gray',
       alpha=0.5,linestyle='--')
ax.xformatter = LONGITUDE_FORMATTER
ax.yformatter = LATITUDE_FORMATTER
ax.set_extent(extent)

In [ ]:
# Coloca um fundo legal com relevo e oceano:
#ax.stock_img()
ax.add_feature(cfeature.BORDERS)
#ax.gridlines()
ax.coastlines(resolution='50m')

Gassian - High Degree

In [ ]:
############### Gassian - High Degree ###############
gaus2 = ax.contourf(lons,lats,mpcalc.smooth_gaussian(vort_850a, 8),levels=10, vmax=-1,cmap='hot')
plt.title('Vorticidade em 850 hPa \n em 01/07/2020, 18 horas\n Gaussian - High Degree')
# Coloca a barra lateral de cores da vorticidade:
plt.colorbar(gaus2)
# Coloca a informacao em um arquivo:
plt.savefig('vort850_gaus2_20200701_18.png')
print('Campo de vorticidade com suavização - Gaussian - High Degree')
#axa[0, 1].imshow(mpcalc.smooth_gaussian(vort_850a, 8), vmax=-1)
#axa[0, 1].set_title('Gaussian - High Degree')
plt.show()

In [ ]:
####Plotar com mais informações geográficas:

# Para colocar os estados no mapa:
estados_br= 'estados_Brasil/estados_2010.shp'
adm1_shapes = list(shpreader.Reader(estados_br).geometries())
# Informações sobre os limites do mapa:
extent = [-60, -30, -20, -50]
# Na projeção PlateCarree:
ax = plt.axes(projection=ccrs.PlateCarree())
ax.add_geometries(adm1_shapes,ccrs.PlateCarree(), edgecolor='black', facecolor='none', alpha=0.5)
ax.add_feature(cfeature.LAND)
ax.add_feature(cfeature.LAKES)
ax.gridlines(crs=ccrs.PlateCarree(),
       draw_labels=True,linewidth=0.5,color='gray',
       alpha=0.5,linestyle='--')
ax.xformatter = LONGITUDE_FORMATTER
ax.yformatter = LATITUDE_FORMATTER
ax.set_extent(extent)
# Coloca um fundo legal com relevo e oceano:
#ax.stock_img()
ax.add_feature(cfeature.BORDERS)
#ax.gridlines()
ax.coastlines(resolution='50m')

Rectangular Smoother

In [ ]:
rec_smo = ax.contourf(lons,lats,mpcalc.smooth_rectangular(vort_850a, (3, 7), 2),levels=10, vmax=-1,cmap='hot')
plt.title('Vorticidade em 850 hPa \n em 01/07/2020, 18 horas\n Rectangular Smoother')
# Coloca a barra lateral de cores da vorticidade:
plt.colorbar(rec_smo)
# Coloca a informacao em um arquivo:
plt.savefig('vort850_rec_smo_20200701_18.png')
print('Campo de vorticidade com suavização - Rectangular Smoother')
#axa[0, 2].imshow(mpcalc.smooth_rectangular(vort_850a, (3, 7), 2), vmax=-1,cmap='hot')
#axa[0, 2].set_title('Rectangular - 3x7 Window\n2 Passes')
plt.show()

In [ ]:
####Plotar com mais informações geográficas:

# Para colocar os estados no mapa:
estados_br= 'estados_Brasil/estados_2010.shp'
adm1_shapes = list(shpreader.Reader(estados_br).geometries())
# Informações sobre os limites do mapa:
extent = [-60, -30, -20, -50]
# Na projeção PlateCarree:
ax = plt.axes(projection=ccrs.PlateCarree())
ax.add_geometries(adm1_shapes,ccrs.PlateCarree(), edgecolor='black', facecolor='none', alpha=0.5)
ax.add_feature(cfeature.LAND)
ax.add_feature(cfeature.LAKES)
ax.gridlines(crs=ccrs.PlateCarree(),
       draw_labels=True,linewidth=0.5,color='gray',
       alpha=0.5,linestyle='--')
ax.xformatter = LONGITUDE_FORMATTER
ax.yformatter = LATITUDE_FORMATTER
ax.set_extent(extent)
# Coloca um fundo legal com relevo e oceano:
#ax.stock_img()
ax.add_feature(cfeature.BORDERS)
#ax.gridlines()
ax.coastlines(resolution='50m')

5-point smoother

In [ ]:
smo_5_1 = ax.contourf(lons,lats,mpcalc.smooth_n_point(vort_850a, 5, 1),levels=10, vmax=-1,cmap='hot')
plt.title('Vorticidade em 850 hPa \n em 01/07/2020, 18 horas\n 5-Point - 1 Pass')
# Coloca a barra lateral de cores da vorticidade:
plt.colorbar(smo_5_1)
# Coloca a informacao em um arquivo:
plt.savefig('vort850_smo_5_1_20200701_18.png')
print('Campo de vorticidade com suavização - 5-Point - 1 Pass')
#axa[1, 0].imshow(mpcalc.smooth_n_point(vort_850a, 5, 1), vmax=-1,cmap='hot')
#axa[1, 0].set_title('5-Point - 1 Pass')
plt.show()

In [ ]:
####Plotar com mais informações geográficas:

# Para colocar os estados no mapa:
estados_br= 'estados_Brasil/estados_2010.shp'
adm1_shapes = list(shpreader.Reader(estados_br).geometries())
# Informações sobre os limites do mapa:
extent = [-60, -30, -20, -50]
# Na projeção PlateCarree:
ax = plt.axes(projection=ccrs.PlateCarree())
ax.add_geometries(adm1_shapes,ccrs.PlateCarree(), edgecolor='black', facecolor='none', alpha=0.5)
ax.add_feature(cfeature.LAND)
ax.add_feature(cfeature.LAKES)
ax.gridlines(crs=ccrs.PlateCarree(),
       draw_labels=True,linewidth=0.5,color='gray',
       alpha=0.5,linestyle='--')
ax.xformatter = LONGITUDE_FORMATTER
ax.yformatter = LATITUDE_FORMATTER
ax.set_extent(extent)
# Coloca um fundo legal com relevo e oceano:
#ax.stock_img()
ax.add_feature(cfeature.BORDERS)
ax.coastlines(resolution='50m')

5-Poit - 4 Pass

In [ ]:
############### 5-Poit - 4 Pass ###############
smo_5_4 = ax.contourf(lons,lats,mpcalc.smooth_n_point(vort_850a, 5, 4),levels=10, vmax=-1,cmap='hot')
plt.title('Vorticidade em 850 hPa \n em 01/07/2020, 18 horas\n 5-Point - 4 Pass')
# Coloca a barra lateral de cores da vorticidade:
plt.colorbar(smo_5_4)
# Coloca a informacao em um arquivo:
plt.savefig('vort850_smo_5_4_20200701_18.png')
print('Campo de vorticidade com suavização - 5-Point - 4 Pass')
#axa[1, 1].imshow(mpcalc.smooth_n_point(vort_850a, 5, 4), vmax=-1,cmap='hot')
#axa[1, 1].set_title('5-Point - 4 Passes')
plt.show()


In [ ]:
####Plotar com mais informações geográficas:

# Para colocar os estados no mapa:
estados_br= 'estados_Brasil/estados_2010.shp'
adm1_shapes = list(shpreader.Reader(estados_br).geometries())
# Informações sobre os limites do mapa:
extent = [-60, -30, -20, -50]
# Na projeção PlateCarree:
ax = plt.axes(projection=ccrs.PlateCarree())
ax.add_geometries(adm1_shapes,ccrs.PlateCarree(), edgecolor='black', facecolor='none', alpha=0.5)
ax.add_feature(cfeature.LAND)
ax.add_feature(cfeature.LAKES)
ax.gridlines(crs=ccrs.PlateCarree(),
       draw_labels=True,linewidth=0.5,color='gray',
       alpha=0.5,linestyle='--')
ax.xformatter = LONGITUDE_FORMATTER
ax.yformatter = LATITUDE_FORMATTER
ax.set_extent(extent)
ax.add_feature(cfeature.BORDERS)
ax.coastlines(resolution='50m')


Circular Smoother

In [ ]:
############### Circular Smoother ###############
circ_2_2 = ax.contourf(lons,lats,mpcalc.smooth_circular(vort_850a, 2, 2),levels=10, vmax=-1,cmap='hot')
plt.title('Vorticidade em 850 hPa \n em 01/07/2020, 18 horas\n Circular - Radius 2, 2 Passes')
# Coloca a barra lateral de cores da vorticidade:
plt.colorbar(circ_2_2)
# Coloca a informacao em um arquivo:
plt.savefig('vort850_circ_2_2_20200701_18.png')
print('Campo de vorticidade com suavização - Circular - Radius2, 2 Passes')
plt.show()

In [ ]:
# Para colocar os estados no mapa:
estados_br= 'estados_Brasil/estados_2010.shp'
adm1_shapes = list(shpreader.Reader(estados_br).geometries())
# Informações sobre os limites do mapa:
extent = [-60, -30, -20, -50]
# Na projeção PlateCarree:
ax = plt.axes(projection=ccrs.PlateCarree())
ax.add_geometries(adm1_shapes,ccrs.PlateCarree(), edgecolor='black', facecolor='none', alpha=0.5)
ax.add_feature(cfeature.LAND)
ax.add_feature(cfeature.LAKES)
ax.gridlines(crs=ccrs.PlateCarree(),
       draw_labels=True,linewidth=0.5,color='gray',
       alpha=0.5,linestyle='--')
ax.xformatter = LONGITUDE_FORMATTER
ax.yformatter = LATITUDE_FORMATTER
ax.set_extent(extent)
# Coloca um fundo legal com relevo e oceano:
#ax.stock_img()
ax.add_feature(cfeature.BORDERS)
#ax.gridlines()
ax.coastlines(resolution='50m')

Arbitrary Window Smoother

In [ ]:
diag_smo = ax.contourf(lons,lats,mpcalc.smooth_window(vort_850a, np.diag(np.ones(5)), 2),levels=10, vmax=-1,cmap='hot')
plt.title('Vorticidade em 850 hPa \n em 01/07/2020, 18 horas\n Diagonal, 2 Passes')
# Coloca a barra lateral de cores da vorticidade:
plt.colorbar(diag_smo)
# Coloca a informacao em um arquivo:
plt.savefig('vort850_diag_smo_20200701_18.png')
print('Campo de vorticidade com suavização - Diagonal, 2 Passes')
plt.show()